In [ ]:
import os 
import re 
from scipy import ndimage, misc 
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import img_to_array


from skimage.transform import resize, rescale
import matplotlib.pyplot as plt
import numpy as np
np. random. seed(0)
import cv2 as cv2

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense ,Conv2D,MaxPooling2D ,Dropout
from tensorflow.keras.layers import Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
import tensorflow as tf

print(tf.__version__)

In [ ]:
coarse= np.load('coarse.npy')*86400
obs= np.load('observed.npy')

In [ ]:
obsflip= np.flip(obs,1)
coarseflip= np.flip(coarse,1)

In [ ]:
for i in range(4):
    a = np.random.randint(0,855)
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.title('High Resolution Imge', color = 'green', fontsize = 20)
    plt.imshow(obsflip[a])
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title('low Resolution Image ', color = 'black', fontsize = 20)
    plt.imshow(coarse[a])
    plt.axis('off')

In [ ]:
high_img= obsflip
low_img= coarse

In [ ]:
high_img.shape

In [ ]:
import random
import cv2

m=high_img.shape[0]
idx= np.arange(0, m)
random.shuffle(idx)

train_idx= idx[0: int(m*0.7)]
valid_idx= idx[len(train_idx): int(m*0.9)]
test_idx= idx[len(train_idx)+len(valid_idx):]

print(len(train_idx), len(valid_idx), len(test_idx))
high_img=obsflip[:,:,:] #reshape(80, 80, 10950)
low_img=coarse[:,:,:]#.reshape(11,11, 10950)

# high_img=np.swapaxes(high_img, 0,2)
# #high_img= np.swapaxes(high_img, 0,1)
# high_img.shape

# low_img=np.swapaxes(low_img, 0,2)
#low_img= np.swapaxes(low_img, 0,1)
# low_img= cv2.resize(low_img, (10950,80, 120))
# low_img.shape

In [ ]:
import cv2

size = 120
height = 80
low_img_stack= np.zeros((len(low_img),  height, size))

for idx in range(len(low_img)):
    img = low_img[idx, :, :]
    img_sm = cv2.resize(img, (size, height), interpolation=cv2.INTER_CUBIC)
    low_img_stack[idx, :, :] = img_sm
low_img_stack.shape

In [ ]:
npad= ((0,0),(40,8),(8,0))
high_img1=np.pad(high_img, pad_width=npad, mode='constant', constant_values=np.nan)
low_img1=np.pad(low_img_stack, pad_width=npad, mode='constant', constant_values=np.nan)
#.shape
print(low_img1.shape, high_img1.shape)

In [ ]:
high_img1.shape

In [ ]:
for i in range(4):
    a= np.random.randint(0,1000)
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.title('High Resolution Imge', color = 'green', fontsize = 20)
    plt.imshow(high_img1[a])
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title('low Resolution Image ', color = 'black', fontsize = 20)
    plt.imshow(low_img1[a])
    plt.axis('off')

In [ ]:
import os

os.mkdir('lowimg')
os.mkdir('highimg')


In [ ]:
high_img2= high_img1.reshape(10950,128,128,1)
low_img2= low_img1.reshape(10950, 128, 128,1)

In [ ]:
high_img1[np.isnan(high_img1)]=-10
low_img1[np.isnan(low_img1)]=-10

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler= MinMaxScaler(feature_range=(0,1))
high_img2= (high_img1- np.min(high_img1) )/(np.max(high_img1)- np.min(high_img1))
low_img2= (low_img1- np.min(low_img1) )/(np.max(low_img1)- np.min(low_img1))

In [ ]:
for i in range(4):
    a= np.random.randint(0,1000)
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.title('High Resolution Imge', color = 'green', fontsize = 20)
    plt.imshow(high_img2[a])
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title('low Resolution Image ', color = 'black', fontsize = 20)
    plt.imshow(low_img2[a])
    plt.axis('off')

In [ ]:
# import matplotlib.image
# for i in range(high_img.shape[2]):
    

#     matplotlib.image.imsave('highimg/{0}.png'.format(i), high_img[:,:,i])
#     matplotlib.image.imsave('lowimg/{0}.png'.format(i), low_img[:,:,i])



In [ ]:
import random

m=high_img1.shape[0]
idx= np.arange(0, m)
random.shuffle(idx)

train_idx= idx[0: int(m*0.7)]
valid_idx= idx[len(train_idx): int(m*0.9)]
test_idx= idx[len(train_idx)+len(valid_idx):]

print(len(train_idx), len(valid_idx), len(test_idx))

In [ ]:
train_high_image = high_img1[train_idx]
train_low_image = low_img1[train_idx]
# train_high_image = np.reshape(train_high_image,(len(train_high_image),SIZE,SIZE,))
# train_low_image = np.reshape(train_low_image,(len(train_low_image),SIZE,SIZE,3))

validation_high_image = high_img1[valid_idx]
validation_low_image = low_img1[valid_idx]
# validation_high_image= np.reshape(validation_high_image,(len(validation_high_image),SIZE,SIZE,3))
# validation_low_image = np.reshape(validation_low_image,(len(validation_low_image),SIZE,SIZE,3))


test_high_image = high_img1[test_idx]
test_low_image = low_img1[test_idx]
# test_high_image= np.reshape(test_high_image,(len(test_high_image),SIZE,SIZE,3))
# test_low_image = np.reshape(test_low_image,(len(test_low_image),SIZE,SIZE,3))

print("Shape of training images:",train_high_image.shape)
print("Shape of test images:",test_low_image.shape)
print("Shape of validation images:",validation_high_image.shape)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model


#SRCNN=tf.keras.models.Sequential([tf.keras.layers.Conv2D(64,9,padding='same',activation='relu'),
 #      tf.keras.layers.Conv2D(64,1,padding='same',activation='relu'),
 #      tf.keras.layers.Conv2D(3,5,padding='same',activation='relu')])

input_img=Input(shape=(128,128,1))
l1=tf.keras.layers.Conv2D(64,9,padding='same',activation='relu') (input_img)
l2=tf.keras.layers.Conv2D(32,1,padding='same',activation='relu') (l1)
l3=tf.keras.layers.Conv2D(1,5,padding='same',activation='relu') (l2)


SRCNN=Model(input_img,l3)


def pixel_mse_loss(x,y):
    return tf.reduce_mean( (x - y) ** 2 )
SRCNN.compile(optimizer=tf.keras.optimizers.Adam(0.001),loss=pixel_mse_loss)
SRCNN.summary()
plot_model(SRCNN, to_file ='super_res.png',show_shapes=True)

In [ ]:
history = SRCNN.fit(train_low_image, train_high_image, epochs = 20, batch_size = 16,
          validation_data = (validation_low_image,validation_high_image))